> 只差代理池啦！！！！！！！！！！！！
> 代理池参考这个
> https://blog.csdn.net/qq_28888837/article/details/88613673

## 在表中获取各个连接

In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.common.proxy import Proxy, ProxyType

In [2]:
# 提供的excel所在路径
file_name='data.xlsx'

In [3]:
data = pd.read_excel(open(file_name, 'rb'), sheet_name='产品目录')
data = data.fillna('')

In [4]:
data.columns

Index(['SKU', '产品名称', '网站标题', '网站链接', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', '网站选项1', '网站选项2', '网站选项3', '网站选项4',
       '网站选项5', '网站选项6', '网站选项7', '网站选项8', '网站选项9', '网站选项10', '网站选项11',
       '网站选项12', '网站选项13', '网站选项14', '网站选项15', '网站选项16', '备注'],
      dtype='object')

In [5]:
sub_data = data[['网站链接', '网站选项1', '网站选项2', '网站选项3','网站选项4','网站选项5','网站选项6','网站选项7','网站选项8','网站选项9','网站选项10','网站选项11','网站选项12','网站选项13','网站选项14','网站选项15','网站选项16','备注']]
sub_data = sub_data.drop_duplicates() # 去重

In [6]:
sub_data.head()

,网站链接,网站选项1,网站选项2,网站选项3,网站选项4,网站选项5,网站选项6,网站选项7,网站选项8,网站选项9,网站选项10,网站选项11,网站选项12,网站选项13,网站选项14,网站选项15,网站选项16,备注
0,https://detail.1688.com/offer/554998071360.htm...,1 混彩色猫眼,4mm,6mm,8mm,10mm,12mm,14mm,16mm,,,,,,,,,
1,https://detail.1688.com/offer/554998071360.htm...,2 深绿色猫眼,4mm,6mm,8mm,10mm,12mm,14mm,16mm,,,,,,,,,
2,https://detail.1688.com/offer/554998071360.htm...,3宝蓝色猫眼石,4mm,6mm,8mm,10mm,12mm,14mm,16mm,,,,,,,,,
3,https://detail.1688.com/offer/554998071360.htm...,4 橘红色猫眼石,4mm,6mm,8mm,10mm,12mm,14mm,16mm,,,,,,,,,
4,https://detail.1688.com/offer/554998071360.htm...,5 白色猫眼石,4mm,6mm,8mm,10mm,12mm,14mm,16mm,,,,,,,,,


In [7]:
# for idx, row in sub_data.iterrows():
#     print(idx)
#     color = row[1]
#     print(color)
#     option = [i for i in row[2:].to_list() if i]
#     print(option)
#     notes = row['备注']
#     print(notes)
#     break
# # 0
# # 1 混彩色猫眼
# # ['4mm', '6mm', '8mm', '10mm', '12mm', '14mm', '16mm', '无']
# # 无

## 获取代理池
https://blog.csdn.net/weixin_44613063/article/details/102538757

> 下面这里暂时用的是免费代理

In [8]:
import requests
from requests.exceptions import ConnectionError
import json
import re
from pyquery import PyQuery as pq

base_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7'
}

def get_page(url, options={}):

    headers = dict(base_headers, **options)
    print('正在抓取', url)
    try:
        response = requests.get(url, headers=headers)
        print('抓取成功', url, response.status_code)
        if response.status_code == 200:
            return response.text
    except ConnectionError:
        print('抓取失败', url)
        return None



# 元类
class ProxyMetaclass(type):
    def __new__(cls, name, bases, attrs):
        count = 0
        attrs['__CrawlFunc__'] = []
        for k, v in attrs.items():
            if 'crawl_' in k:
                attrs['__CrawlFunc__'].append(k)
                count += 1
        attrs['__CrawlFuncCount__'] = count
        return type.__new__(cls, name, bases, attrs)

class Crawler(object, metaclass=ProxyMetaclass):
    def get_proxies(self, callback):
        proxies = []
        for proxy in eval("self.{}()".format(callback)):
            print('成功获取到代理', proxy)
            proxies.append(proxy)
        return proxies

    def crawl_daili66(self, page_count=4):
        start_url = 'http://www.66ip.cn/{}.html'
        urls = [start_url.format(page) for page in range(1, page_count + 1)]
        for url in urls:
            print('Crawling', url)
            html = get_page(url)
            if html:
                doc = pq(html)
                trs = doc('.containerbox table tr:gt(0)').items()
                for tr in trs:
                    ip = tr.find('td:nth-child(1)').text()
                    port = tr.find('td:nth-child(2)').text()
                    yield ':'.join([ip, port])

    def crawl_xicidaili(self):
        for i in range(1, 3):
            start_url = 'http://www.xicidaili.com/nn/{}'.format(i)
            headers = {
                'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                'Host':'www.xicidaili.com',
                'Referer':'http://www.xicidaili.com/nn/3',
                'Upgrade-Insecure-Requests':'1',
            }
            html = get_page(start_url, options=headers)
            if html:
                find_trs = re.compile('<tr class.*?>(.*?)</tr>', re.S)
                trs = find_trs.findall(html)
                for tr in trs:
                    find_ip = re.compile('<td>(\d+\.\d+\.\d+\.\d+)</td>') 
                    re_ip_address = find_ip.findall(tr)
                    find_port = re.compile('<td>(\d+)</td>')
                    re_port = find_port.findall(tr)
                    for address,port in zip(re_ip_address, re_port):
                        address_port = address+':'+port
                        yield address_port.replace(' ','')

    def crawl_ip3366(self):
        for i in range(1, 4):
            start_url = 'http://www.ip3366.net/?stype=1&page={}'.format(i)
            html = get_page(start_url)
            if html:
                find_tr = re.compile('<tr>(.*?)</tr>', re.S)
                trs = find_tr.findall(html)
                for s in range(1, len(trs)):
                    find_ip = re.compile('<td>(\d+\.\d+\.\d+\.\d+)</td>')
                    re_ip_address = find_ip.findall(trs[s])
                    find_port = re.compile('<td>(\d+)</td>')
                    re_port = find_port.findall(trs[s])
                    for address,port in zip(re_ip_address, re_port):
                        address_port = address+':'+port
                        yield address_port.replace(' ','')

    def crawl_kuaidaili(self):
        for i in range(1, 4):
            start_url = 'http://www.kuaidaili.com/free/inha/{}/'.format(i)
            html = get_page(start_url)
            if html:
                ip_address = re.compile('<td data-title="IP">(.*?)</td>') 
                re_ip_address = ip_address.findall(html)
                port = re.compile('<td data-title="PORT">(.*?)</td>')
                re_port = port.findall(html)
                for address,port in zip(re_ip_address, re_port):
                    address_port = address+':'+port
                    yield address_port.replace(' ','')
import sys

class Getter():
    def __init__(self):
        self.crawler = Crawler()
        self.ips = []
    
    def run(self):
        print('获取器开始执行')
        for callback_label in range(self.crawler.__CrawlFuncCount__):
            callback = self.crawler.__CrawlFunc__[callback_label]
            # 获取代理
            all_ip = self.crawler.get_proxies(callback)
            self.ips += all_ip

In [25]:
get = Getter()
get.run()

获取器开始执行
Crawling http://www.66ip.cn/1.html
正在抓取 http://www.66ip.cn/1.html
抓取成功 http://www.66ip.cn/1.html 200
成功获取到代理 103.233.198.156:8081
成功获取到代理 46.166.151.198:5836
成功获取到代理 175.43.58.44:9999
成功获取到代理 47.104.255.144:8118
成功获取到代理 183.166.110.167:9999
成功获取到代理 181.225.107.100:999
成功获取到代理 49.231.140.120:8080
成功获取到代理 202.29.220.34:49350
Crawling http://www.66ip.cn/2.html
正在抓取 http://www.66ip.cn/2.html
抓取成功 http://www.66ip.cn/2.html 200
成功获取到代理 117.69.12.106:9999
成功获取到代理 45.230.8.20:51200
成功获取到代理 94.23.30.184:3128
成功获取到代理 61.118.35.94:55725
成功获取到代理 183.166.102.16:9999
成功获取到代理 125.108.125.179:9000
成功获取到代理 110.243.29.54:9999
成功获取到代理 139.255.25.83:3128
成功获取到代理 110.243.22.136:9999
成功获取到代理 85.175.216.32:53281
成功获取到代理 123.163.115.139:9999
成功获取到代理 171.35.162.196:9999
Crawling http://www.66ip.cn/3.html
正在抓取 http://www.66ip.cn/3.html
抓取成功 http://www.66ip.cn/3.html 200
成功获取到代理 36.255.84.213:84
成功获取到代理 163.204.245.131:9999
成功获取到代理 123.55.5.112:9999
成功获取到代理 41.60.216.199:8080
成功获取到代理 175.43.59.39:9999
成功

In [26]:
print(get.ips)

['103.233.198.156:8081', '46.166.151.198:5836', '175.43.58.44:9999', '47.104.255.144:8118', '183.166.110.167:9999', '181.225.107.100:999', '49.231.140.120:8080', '202.29.220.34:49350', '117.69.12.106:9999', '45.230.8.20:51200', '94.23.30.184:3128', '61.118.35.94:55725', '183.166.102.16:9999', '125.108.125.179:9000', '110.243.29.54:9999', '139.255.25.83:3128', '110.243.22.136:9999', '85.175.216.32:53281', '123.163.115.139:9999', '171.35.162.196:9999', '36.255.84.213:84', '163.204.245.131:9999', '123.55.5.112:9999', '41.60.216.199:8080', '175.43.59.39:9999', '61.8.78.130:8080', '1.199.30.248:9999', '163.204.242.92:9999', '94.153.169.22:59177', '163.125.158.102:8888', '185.238.239.72:8090', '103.114.10.177:8080', '91.207.184.105:53281', '123.55.101.114:9999', '123.163.118.6:9999', '183.89.64.243:8080', '123.163.118.25:9999', '181.118.159.30:37399', '71.183.100.76:42413', '62.171.130.72:3128', '113.120.60.163:9999', '131.196.141.222:33729', '5.59.141.183:61981', '103.14.234.22:8080', '36.2

> 这里采用淘宝买的代理池

## 获得链接后用爬虫爬取

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [12]:
# url = 'https://detail.1688.com/offer/554998071360.html?spm=a2615.7691456.autotrace-offerGeneral.7.252261a8RKNvGr'
# browser.get(url)
# browser.implicitly_wait(2)
# browser.close()

In [13]:
# browser.find_element_by_css_selector('#mod-detail-bd > div.region-custom.region-detail-property.region-takla.ui-sortable.region-vertical > div.widget-custom.offerdetail_ditto_purchasing > div > div > div > div.obj-leading > div.obj-content > ul > li:nth-child(2) > div > a').click()

In [41]:


# 说明：status_dict记录货品是否有货
# status_dict[货品名]为一个字典
# status_dict[货品名][颜色名] = 1 if 有货 else 0
status_dict = {}

proxy_id = 0

for idx, row in sub_data.iterrows():
    # 获取该行对应的颜色
    color = row[1]
    status_dict[color] = {}
    # 获取对应颜色下各个型号
    options = [i for i in row[2:].to_list() if i]
    notes = row['备注']
    
#     print(color)
#     print(options)
#     print(notes)

    # 设置代理
    # 循环设置，避免溢出
#     if idx and idx % 5 == 0:
#         time.sleep(10)
#     PROXY=get.ips[proxy_id]
#     print(PROXY)
#     proxy_id+=1
#     proxy_id = proxy_id % len(get.ips)
    PROXY='127.0.0.1:10889'
    
    # 浏览器爬取
    chrome_options = Options()
#     chrome_options.add_argument('--headless')
#     chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--proxy-server=http://%s' % PROXY)
    chrome_options.add_argument('--proxy-server=https://%s' % PROXY)
    
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe', options=chrome_options)
    browser.delete_all_cookies()
    browser.maximize_window()
    # 进入网站
    url = row['网站链接']
    browser.get(url)
    # 等待加载
    browser.implicitly_wait(10)

    # 获取页面中各个元素
#     alters = browser.find_elements_by_class_name('image')
    # 这个获取的部分可能随时会变
    alters = browser.find_elements_by_css_selector('#mod-detail-bd > div.detail-v2018-layout-left > div.region-custom.region-detail-property.region-takla.ui-sortable.region-vertical.dsc-version2018-page-fix-content-mid > div.widget-custom.offerdetail_ditto_purchasing > div > div > div > div.obj-leading > div.obj-content > ul > li > div > a')
    print(alters)
    for alter in alters:
        _color = alter.get_attribute('title')
        print(alters, _color)
        # 如果当前行的颜色匹配，则点击，否则跳过
        if color != _color:
            continue
        print(_color)

        try:
            alter.click()
        except Exception as e:
            print("Error ", _color)
            continue

        # 得到当前颜色下各个型号
        _options = browser.find_elements_by_css_selector('#mod-detail-bd > div.detail-v2018-layout-left > div.region-custom.region-detail-property.region-takla.ui-sortable.region-vertical.dsc-version2018-page-fix-content-mid > div.widget-custom.offerdetail_ditto_purchasing > div > div > div > div.obj-sku > div.obj-content > table > tbody > tr')
#         _options = browser.find_elements_by_css_selector('#mod-detail-bd > div.region-custom.region-detail-property.region-takla.ui-sortable.region-vertical > div.widget-custom.offerdetail_ditto_purchasing > div > div > div > div.obj-sku > div.obj-content > table > tbody > tr')
        # 遍历各个型号
        for _option in _options:
            # 为空则退出
            if not _option.text: 
                continue
            _option = (_option.text.strip('\n-\n+').split(' '))
            # 不在选项中则退出
            __op = _option[0]
            __num = _option[3]
            
            if __op not in options:
                continue
#             print(__op, __num[:-3])
            
            if __num[-3:] != '条可售' or int(__num[:-3]) <= 0:
                # 记录无货信息
                status_dict[color][__op] = 0
            else:
                # 记录有货信息
                status_dict[color][__op] = int(__num[:-3])
        
        
        # 新增：根据备注来进行调节
        # 如果备注否，则说明全部没有货，显示有误
        # 如果备注具体型号，则具体型号没有货
        if notes == '否':
            for option in options:
                status_dict[color][option] = 0
            continue
        elif notes != '':
            empty_options = notes.split(',')
            for option in empty_options:
                option = option.strip(' ')
                status_dict[color][option] = 0
        print(status_dict[color])
        time.sleep(1)
    browser.close()
    break

[]


In [34]:
print(status_dict)

{'1 混彩色猫眼': {}}


## 取得货品状态后对原excel表格进行更新

In [16]:
# 对表中颜色进行分析
import openpyxl
from openpyxl.styles import PatternFill, colors

work_book = openpyxl.load_workbook(file_name)
work_sheet = work_book.get_sheet_by_name('产品目录')
print(work_sheet.max_row,work_sheet.max_column)

184 31


<ipython-input-16-f3ae4ef7ebbb>:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  work_sheet = work_book.get_sheet_by_name('产品目录')


In [17]:
fill_red = PatternFill('solid', fgColor='ff0000')
fill_blue = PatternFill('solid', fgColor='0000ff')
fill_yellow = PatternFill('solid', fgColor='ffff00')
fill_white = PatternFill('none', fgColor='ffffff')
isFirstTime = True

pre_color = {}

# 遍历提供的表格并进行修改
for i in range(1, work_sheet.max_row):
    # 当前行对应的颜色
    row_color = ""
    for j in range(1, work_sheet.max_column):
        _cell = (work_sheet.cell(i, j))
        if not _cell.value:
            continue
        # 如果当前颜色在我们爬取结果中，则记录当前行的颜色
        if _cell.value in status_dict:
            row_color = _cell.value
        
        # 根据检测之前的颜色和已有状态来变色
        try:
            # 上一次为蓝色的情况
            if _cell.fill == fill_blue:
                if _cell.value in status_dict[row_color].keys() and status_dict[row_color][_cell.value]:
                    _cell.fill = fill_yellow
                else:
                    _cell.fill = fill_red
            # 上一次为黄色的情况
            elif _cell.fill == fill_yellow:
                if _cell.value in status_dict[row_color].keys() and status_dict[row_color][_cell.value]:
                    _cell.fill = fill_white
                else:
                    _cell.fill = fill_blue
            
            # 上一次为红色的情况
            elif _cell.fill == fill_red:
                if _cell.value in status_dict[row_color].keys() and status_dict[row_color][_cell.value]:
                    _cell.fill = fill_yellow
                else:
                    _cell.fill = fill_red
            # 上一次为无色的情况
            else:
                if _cell.value in status_dict[row_color].keys() and status_dict[row_color][_cell.value]:
                    pass
                else:
                    # 如果是第一次，无货则直接置为红色
                    if isFirstTime:
                        _cell.fill = fill_red
                    else:
                        _cell.fill = fill_blue
        except Exception as e:
            pass

work_book.save(file_name)

PermissionError: [Errno 13] Permission denied: 'data.xlsx'